In [1]:
import pandas as pd
import numpy as np
import os
print("pandas version : ",pd.__version__)
print("numpy version : ", np.__version__)

pandas version :  1.0.5
numpy version :  1.19.1


In [3]:
data = pd.read_csv("data/new_performance_ver3.csv")
names = data.columns
for name in names:
    if "Unnamed" in name:
        data.pop(name)
data.head()

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,요일,IsHoliday
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01,1,06:00:00,1.0,1.0
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01,1,06:00:00,1.0,1.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,2019-01-01,1,06:20:00,1.0,1.0
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,2019-01-01,1,06:20:00,1.0,1.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,2019-01-01,1,06:40:00,1.0,1.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35379 entries, 0 to 35378
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   방송일시       35379 non-null  object 
 1   노출(분)      35379 non-null  float64
 2   마더코드       35379 non-null  int64  
 3   상품코드       35379 non-null  int64  
 4   상품명        35379 non-null  object 
 5   상품군        35379 non-null  object 
 6   판매단가       35379 non-null  int64  
 7   취급액        35379 non-null  float64
 8   date       35379 non-null  object 
 9   week       35379 non-null  int64  
 10  time       35379 non-null  object 
 11  요일         1324 non-null   float64
 12  IsHoliday  35379 non-null  float64
dtypes: float64(4), int64(4), object(5)
memory usage: 3.5+ MB


요일에는 null값이 왜이렇게 많을까요..?

In [5]:
data.isna().sum()

방송일시             0
노출(분)            0
마더코드             0
상품코드             0
상품명              0
상품군              0
판매단가             0
취급액              0
date             0
week             0
time             0
요일           34055
IsHoliday        0
dtype: int64

In [6]:
weather = pd.read_csv("data/지역별가중합날씨.csv")
weather.head()

,time,TEMP,HUM
0,2019-01-01 01:00,-5.231287,53.536734
1,2019-01-01 02:00,-5.662894,56.822305
2,2019-01-01 03:00,-6.029719,58.833658
3,2019-01-01 04:00,-6.343332,60.112326
4,2019-01-01 05:00,-6.590452,61.873758


In [7]:
type(data.date[0])

str

In [8]:
def hoursplit(data):
    return data[:2]
def minsplit(data):
    return data[3:5]

In [9]:
data1 = data.copy()
data1['hour'] = data['time'].apply(lambda x: hoursplit(x))
data1['min'] = data['time'].apply(lambda x: minsplit(x))
data1.head(2)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,요일,IsHoliday,hour,min
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01,1,06:00:00,1.0,1.0,06,00
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01,1,06:00:00,1.0,1.0,06,00


In [10]:
from datetime import datetime

## Merging

data1과 weather를 날짜와 '시'로 merge해주기 위해 weather의 time컬럼을 date와 hour로 나누어 주는 작업을 진행했습니다.

- datetime모듈의 strptime과 strftime을 이중으로 lamda함수에 넣어주었습니다.
- 컬럼 명을 data1과 동일하게 date, hour로 설정하였습니다.

In [11]:
weather['real_date'] = weather['time'].apply(lambda x: datetime.strptime(x,
                                                                   '%Y-%m-%d %H:%M').strftime("%Y-%m-%d"))
weather['hour'] = weather['time'].apply(lambda x: datetime.strptime(x,
                                                                   '%Y-%m-%d %H:%M').strftime("%H"))
weather = weather.drop(['time'], axis=1)

weather.head()


,TEMP,HUM,real_date,hour
0,-5.231287,53.536734,2019-01-01,01
1,-5.662894,56.822305,2019-01-01,02
2,-6.029719,58.833658,2019-01-01,03
3,-6.343332,60.112326,2019-01-01,04
4,-6.590452,61.873758,2019-01-01,05


동일한 hour를 가진 행에는 동일한 날씨 정보가 삽입 되었습니다. 근데 date는 0시는 전날로 들어감..!! 1월 정보랑 방송 일시 기준 날짜를 다시 넣어야 할듯!!

In [12]:
data1['real_date'] = data1['방송일시'].apply(lambda x: datetime.strptime(x,
                                                                   '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d"))
data1.head(2)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,요일,IsHoliday,hour,min,real_date
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01,1,06:00:00,1.0,1.0,06,00,2019-01-01
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01,1,06:00:00,1.0,1.0,06,00,2019-01-01


- date와 상품코드 이용해서 단일 방송 구하기(groupby)

In [13]:
df = pd.merge(data1, weather, on = ['real_date', 'hour'])
display(df.head(2))
display(df.tail(2))

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,요일,IsHoliday,hour,min,real_date,TEMP,HUM
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,2019-01-01,1,06:00:00,1.0,1.0,06,00,2019-01-01,-6.720402,63.476553
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,2019-01-01,1,06:00:00,1.0,1.0,06,00,2019-01-01,-6.720402,63.476553


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,date,week,time,요일,IsHoliday,hour,min,real_date,TEMP,HUM
35282,2019-12-31 00:40:00,20.0,100320,201300,올바로 문어발 샤워기 1+1,생활용품,54900,23852000.0,2019-12-30,0,00:40:00,NaN,0.0,00,40,2019-12-31,-1.885042,59.789544
35283,2019-12-31 00:40:00,20.0,100320,201301,올바로 문어발 샤워기 1세트,생활용품,28900,9273000.0,2019-12-30,0,00:40:00,NaN,0.0,00,40,2019-12-31,-1.885042,59.789544


Merge 과정에서 1월 1일 정보가 drop되어 2020.1 ~ 2020.6의 날씨 정보를 업데이트 해야할 것 같습니다. 200개 정도가 na값으로 처리된 것으로 보입니다.

In [ ]:
df.info()

## 방송 초/중/후반 정보 삽입

- 방송일시와 마더코드 사용
    - 동일한 방송일시와 마더코드는 한 타임의 방송이라는 뜻
    - 노출(분)의 각 시간대 unique값을 합해서 각각 초반/중반/후반대로 나누어 줌
- 현재 익일 새벽 2시까지의 방송은 전날 방송에 포함되는 것으로 판단하고 있음.
    - zapping time, 혹은 다른 이유로 애매하게 당일과 익일에 방송 시간이 걸쳐있을 경우 계산에 오류가 발생할 수 있음.
    - 하루에 여러번 같은 상품?/내용의 방송을 할 수 있지 않음?
    - 방송일시, 마터코드, date까지 이용
- datetime 모듈을 사용하여 시간 차이를 구함
    - 그냥 빼기 연산을 해주면 되는데 이 때, return값은 timedelta클래스로 반환됨
    - **timedelta** : days, seconds, miliseconds를 통해 시간 차의 일 수, 초, 밀리 초에 접근 가능함. 3600으로 나누면 '시간'으로 접근할 수 있음.

In [ ]:
def stage_transformer(real_time, code, date):
    